In [38]:
from fv3fit.reservoir import (
    ReservoirHyperparameters,
    Reservoir, 
    RankDivider,
)

from fv3fit.reservoir.transformers import DoNothingAutoencoder
from fv3fit.reservoir.domain2 import OverlapRankXYDivider

import numpy as np
from importlib import reload

import fv3fit.reservoir.readout as readout

In [4]:
autoencoder = DoNothingAutoencoder([79])

In [6]:

hyperparameters = ReservoirHyperparameters(
    state_size=3000,
    adjacency_matrix_sparsity=0.9,
    spectral_radius=0.5,
    input_coupling_sparsity=0.5,
    input_coupling_scaling=0.1,
)

rank_divider = RankDivider([8, 8], ["x", "y"], [52, 52, 79], overlap=2)


In [7]:
rank_divider.subdomain_size_with_overlap

7900

In [8]:
input_size_subdomain = rank_divider.subdomain_size_with_overlap * rank_divider.n_subdomains
output_size_subdomain = rank_divider.subdomain_xy_size_without_overlap ** 2 * 79 * rank_divider.n_subdomains

In [9]:
input_size_subdomain * output_size_subdomain * 8 / 1024**3

685.65673828125

685 GB for single C48 model tile.... with full readout matrix resolution. Need to stack matrices in sparse matrix to get a true readout layer.  But! We have a combine_readouts that loads the block diagonal matrix into

In [10]:
rank_divider.get_subdomain_extent(True)

(10, 10, 79)

### An aside working through shapes of everything

readout takes flat vector of subdomains stacked column by column

input @ coefficenits

7900 <> flattened state x 1 ?? @ 

7900 x ns
hidden_state x ns

(7900 + hidden_state) x ns
nf

nf*ns 1D array

What are the readout weights?

Ax = y

nf*ns x nf*ns

training inputs X is ntimes x nf*ns


Ridge regression

We have a matrix of input X and a matrix of output Y.  We want to use regression to find a matrix W such that Y = XW.  This is a linear regression problem.  Ridge regression is a regularized version of linear regression.  The regularization term is a penalty on the size of the coefficients.  The ridge coefficients minimize a penalized residual sum of squares,

If X is nsamples by nfeatures, then W is nfeatures by noutputs

W = (X^T X + alpha*I)^-1 X^T Y

X^T X is nfeatures x nfeatures

W is nfeatures x noutputs

XW is nsamples x noutputs

Readouts are

XW + b = y

for each subdomain

if I stack subdomains

* W (subdomain, nfeatures, noutputs)
* X (subdomain, (sample?), nfeatures)
* y = (subdomain, (sample?), noutputs)

Subdomains leak out into the training quite a bit.  Can I contain it?

`OverlapRankDivider` is some work to remove most of special care for subdomains.

In [11]:
divider = OverlapRankXYDivider((4, 4), (52, 52), overlap=2, z_feature=autoencoder.n_latent_dims)

In [12]:
divider.flat_subdomain_shape[0]

20224

In [13]:
reservoir = Reservoir(hyperparameters, input_size=divider.flat_subdomain_shape[0])

In [14]:
reservoir.reset_state((divider.n_subdomains, divider.flat_subdomain_shape[0]))

In [15]:
reservoir.state.shape

(16, 3000)

In [16]:
reservoir.W_in.T.shape

(20224, 3000)

In [20]:
test_input = np.random.rand(divider.n_subdomains, divider.flat_subdomain_shape[0])
test_input.shape

(16, 20224)

In [21]:
reservoir.reset_state(test_input.shape)
reservoir.state.shape

(16, 3000)

In [22]:
reservoir.synchronize(np.array([test_input]*5))

In [34]:
coo_W_in = reservoir.W_in.tocoo()
csc_W_in = reservoir.W_in.tocsc()
csr_W_in = reservoir.W_in.tocsr()

In [36]:
coo_W_in.shape

(3000, 20224)

In [37]:
%%timeit -r 3 -n 5
test_input @ coo_W_in.T

1 s ± 749 µs per loop (mean ± std. dev. of 3 runs, 5 loops each)


In [25]:
%%timeit -r 3 -n 5
test_input @ csc_W_in.T

262 ms ± 340 µs per loop (mean ± std. dev. of 3 runs, 5 loops each)


In [26]:
%%timeit -r 3 -n 5
test_input @ csr_W_in.T

330 ms ± 2.15 ms per loop (mean ± std. dev. of 3 runs, 5 loops each)


In [27]:
csc_W_in.data.nbytes / 1024**2

231.4453125

In [28]:
default = csr_W_in.toarray()

In [29]:
%%timeit -r 3 -n 5
test_input @ default.T

89.2 ms ± 2.56 ms per loop (mean ± std. dev. of 3 runs, 5 loops each)


In [30]:
default.nbytes / 1024**2

462.890625

In [31]:
test_input.shape

(16, 20224)

In [32]:
reservoir.W_res = reservoir.W_res.tocsc()
reservoir.W_in = reservoir.W_in.tocsc()

In [33]:
%%timeit -r 1 -n 1
reservoir.synchronize(np.array([test_input]*25))

6.8 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


Using `csc` we reduce the increment time of generating the state timeseries by 75%.  Best would be just to use a regular matrix (90% reduction), since I think 200 MB vs. 500 MB is not a big deal?

## Regressor Checks

In [39]:
reg_hypers = readout.BatchLinearRegressorHyperparameters(l2=0.001, add_bias_term=True)
regressor = readout.BatchLinearRegressor(reg_hypers)

In [40]:
X = np.arange(12).reshape(6, 2)
y = np.concatenate([
    2 * X[:, 0:1] + 3 * X[:, 1:2],
    4 * X[:, 0:1] + 1 * X[:, 1:2],
    5 * X[:, 0:1] + 6 * X[:, 1:2],
], axis=-1)

In [41]:
print(X.shape)
print(y.shape)

(6, 2)
(6, 3)


In [42]:
for i in range(0, 6, 2):
    regressor.batch_update(X[i : i + 2], y[i : i + 2])

In [43]:
coefs, intercept = regressor.get_weights()

In [44]:
res = X @ coefs + intercept

### Add subdomain leading dimension

In [45]:
coefs_sub = np.array([coefs]*4)

In [46]:
coefs_sub.shape

(4, 2, 3)

In [47]:
X_sub = np.array([X]*4)

In [48]:
X_sub.shape

(4, 6, 2)

In [49]:
res_sub = X_sub @ coefs_sub + intercept
np.testing.assert_allclose(res_sub[0], res)

Dot product along trailing axis and 2nd-to-last trailing axes works with leading subdomain.  Stacking the coefficients and intercepts should be OK.  Need to fit separately? Or just form A, B separately?

Note the following section was done adjusting git weights to allow a leading subdomain dimension for W.  It didn't end up making a difference so I removed.  Showing the timing below.

In [50]:
y_sub = np.array([y]*4)

In [51]:
print(X_sub.shape, y_sub.shape)

(4, 6, 2) (4, 6, 3)


In [52]:
joined_A = np.array([regressor.A]*4)
joined_B = np.array([regressor.B]*4)

In [53]:
new_regressor = readout.BatchLinearRegressor(reg_hypers)

In [54]:
new_regressor.A = joined_A
new_regressor.B = joined_B

I could combine the weight calculations, but it might break the least_squares solve.  Since there's fewer subdomains, it might not be that large of a difference timing-wise.

In [150]:
test_XtX = np.random.rand(5_000, 5_000)
test_Xty = np.random.rand(5_000, 500)

regressors = []
for i in range(16):
    reg = readout.BatchLinearRegressor(reg_hypers)
    reg.A = test_XtX
    reg.B = test_Xty
    regressors.append(reg)

In [151]:

%%timeit -r 1 -n 1

subdomain_readout_coeffs = []
subdomain_intercepts = []
for r, regressor in enumerate(regressors):
    coefs_, intercepts_ = regressor.get_weights()
    subdomain_readout_coeffs.append(coefs_)
    subdomain_intercepts.append(intercepts_)

28.7 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [154]:
%%timeit -r 1 -n 1

new_regressor = readout.BatchLinearRegressor(reg_hypers)
new_regressor.A = np.array([test_XtX]*16)
new_regressor.B = np.array([test_Xty]*16)
new_regressor.get_weights()

29.3 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


Not any faster, so leave it as is.